In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-20 01:26:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-12-20 01:26:17 (6.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# start a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Challenge16-D2-VineReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [120]:
## load amazon data to df
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [121]:
vine_df = df.select(['review_id', 'star_rating','helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.printSchema()
vine_df.show(5)

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# recreate vine_table 

In [122]:
# 1. vine_table that has reviews with 20 more votes
vine_table = vine_df.filter('total_votes>=20')
vine_table.show(5)
print(f'There are {vine_table.count()} reviews that have more than 20 votes.')

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

There are 7055 reviews that have more than 20 votes.


In [123]:
# 2. a table where number of helpful_votes/total_votes >= 50%
helpful_vote_df = vine_table.filter('helpful_votes/total_votes>=0.5')

helpful_vote_df.show(5)
print(f'There are { helpful_vote_df.count()} reviews that have more than 50% helpful votes.')

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

There are 6711 reviews that have more than 50% helpful votes.


In [124]:
# 3. table for Vine users
vine_y_df = helpful_vote_df.filter(helpful_vote_df['vine'] == 'Y')

vine_y_df.show(5)
print(f'There are {vine_y_df.count()} reviews from Vine users.')

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RPJ7C4HP3BHXN|          5|           25|         29|   Y|                N|
|R3SB08XK0M7993|          4|           44|         49|   Y|                N|
|R26X1DACMCN3YF|          3|           16|         21|   Y|                N|
|R17HKKDE9ZRVOO|          4|           31|         35|   Y|                N|
|R1R7GNSCMO8U7I|          4|           24|         28|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

There are 21 reviews from Vine users.


In [125]:
# 4. vine=N
vine_n_df = helpful_vote_df.filter(helpful_vote_df['vine'] == 'N')
vine_n_df.show(5)
print(f'There are {vine_n_df.count()} helpful reviews from non-Vine users.')

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

There are 6690 helpful reviews from non-Vine users.


In [151]:
# 5. total # votes, #-5-stars, % 5-stars
grouped_df = helpful_vote_df.groupby('vine').sum('total_votes', 'helpful_votes')
grouped_df.show()
# total numbers of reviews per user type
helpful_vote_df.groupby('vine').count().show()

+----+----------------+------------------+
|vine|sum(total_votes)|sum(helpful_votes)|
+----+----------------+------------------+
|   Y|             932|               870|
|   N|          387919|            362765|
+----+----------------+------------------+

+----+-----+
|vine|count|
+----+-----+
|   Y|   21|
|   N| 6690|
+----+-----+



In [130]:
# total numbers of reviews per user type
helpful_vote_df.groupby('vine').count().show()

+----+-----+
|vine|count|
+----+-----+
|   Y|   21|
|   N| 6690|
+----+-----+



In [131]:
# total 5-star helpful reviews
helpful_vote_df.filter('star_rating==5').count()

3458

In [132]:
# total # of 5-star reviews from Vine users
helpful_vote_df.filter((helpful_vote_df.star_rating==5) & (helpful_vote_df.vine == 'Y')).count()

10

In [133]:
# total # of 5-star reviews from non-vine users
helpful_vote_df.filter((helpful_vote_df.star_rating==5) & (helpful_vote_df.vine == 'N')).count()

3448

In [134]:
y_5star = vine_y_df.filter("star_rating==5").count()
n_5star = vine_n_df.filter("star_rating==5").count()

print(f' total 5-star Vine reviews: {y_5star} ')
print(f' total 5-star non-Vine reviews: {n_5star}')

 total 5-star Vine reviews: 10 
 total 5-star non-Vine reviews: 3448


In [135]:
y_5star_percentage = y_5star/6711 *100
print(f'Percentage of 5-star reviews for Vine users: {y_5star_percentage:.2f}%')

Percentage of 5-star reviews for Vine users: 0.15%


In [136]:
n_5star_percentage = n_5star/6711*100
print(f'Percentage of 5-star reviews for Vine users: {n_5star_percentage:.2f}%')

Percentage of 5-star reviews for Vine users: 51.38%


In [141]:
fivestars_df= helpful_vote_df.filter('star_rating==5')
fivestars_df.show(5)
fivestars_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



3458

In [144]:
# among 5-star reviews, 
fivestars_df.groupby('verified_purchase').count().show()

+-----------------+-----+
|verified_purchase|count|
+-----------------+-----+
|                Y| 2726|
|                N|  732|
+-----------------+-----+



In [150]:
helpful_vote_df.groupby('verified_purchase').count().show()

+-----------------+-----+
|verified_purchase|count|
+-----------------+-----+
|                Y| 5240|
|                N| 1471|
+-----------------+-----+



In [149]:
helpful_vote_df.groupby('vine').agg({'verified_purchase':'count'}).show()

+----+------------------------+
|vine|count(verified_purchase)|
+----+------------------------+
|   Y|                      21|
|   N|                    6690|
+----+------------------------+



In [138]:
helpful_vote_df.groupby('star_rating').count().show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 1066|
|          3|  635|
|          5| 3458|
|          4| 1078|
|          2|  474|
+-----------+-----+



In [152]:
# result tables

#grouped_df = helpful_vote_df.groupby('vine').sum('total_votes', 'helpful_votes')
grouped_df.show()
helpful_vote_df.groupby('vine').count().show()
helpful_vote_df.groupby('verified_purchase').count().show()

+----+----------------+------------------+
|vine|sum(total_votes)|sum(helpful_votes)|
+----+----------------+------------------+
|   Y|             932|               870|
|   N|          387919|            362765|
+----+----------------+------------------+

+----+-----+
|vine|count|
+----+-----+
|   Y|   21|
|   N| 6690|
+----+-----+

+-----------------+-----+
|verified_purchase|count|
+-----------------+-----+
|                Y| 5240|
|                N| 1471|
+-----------------+-----+



In [153]:
# screenshot 2
print(f'There are {vine_table.count()} reviews that have more than 20 votes.')
print(f'There are { helpful_vote_df.count()} reviews that have more than 50% helpful votes.')
print(f'There are {vine_y_df.count()} reviews from Vine users.')
print(f'There are {vine_n_df.count()} helpful reviews from non-Vine users.')
print(f' total 5-star Vine reviews: {y_5star} ')
print(f' total 5-star non-Vine reviews: {n_5star}')
print(f'Percentage of 5-star reviews for Vine users: {y_5star_percentage:.2f}%')
print(f'Percentage of 5-star reviews for Vine users: {n_5star_percentage:.2f}%')

There are 7055 reviews that have more than 20 votes.
There are 6711 reviews that have more than 50% helpful votes.
There are 21 reviews from Vine users.
There are 6690 helpful reviews from non-Vine users.
 total 5-star Vine reviews: 10 
 total 5-star non-Vine reviews: 3448
Percentage of 5-star reviews for Vine users: 0.15%
Percentage of 5-star reviews for Vine users: 51.38%


In [ ]:
# further analysis: review text 